In [1]:
import pandas as pd
import numpy as np
import re
import openai
import os
from textreader import scrapData

In [2]:
people = [x for x in os.listdir(f"./dataset/") if not x.startswith('.') and x != "outputs"]
characters = {}

dfs = {}
for person in people:
    chats = os.listdir(f"./dataset/{person}")
    characters[person] = chats
    dfs[person] = pd.DataFrame({"input_text":[], "output_text":[]})

    for chat in chats:
        scrapData(chat[:-4], f"./dataset/{person}/{chat}")
        
        newDf = pd.read_json(f"./dataset/outputs/{chat[:-4]}_Output.json")
        dfs[person] = dfs[person].append(newDf, ignore_index=True)
        
dfs

1042
1043
1437
1438


/var/folders/2j/0_wrpd3n1xs8vhzsjgsnfdhm0000gn/T/ipykernel_35572/394106510.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfs[person] = dfs[person].append(newDf, ignore_index=True)
/var/folders/2j/0_wrpd3n1xs8vhzsjgsnfdhm0000gn/T/ipykernel_35572/394106510.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfs[person] = dfs[person].append(newDf, ignore_index=True)


2701
2702


/var/folders/2j/0_wrpd3n1xs8vhzsjgsnfdhm0000gn/T/ipykernel_35572/394106510.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfs[person] = dfs[person].append(newDf, ignore_index=True)


3968
3967


/var/folders/2j/0_wrpd3n1xs8vhzsjgsnfdhm0000gn/T/ipykernel_35572/394106510.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfs[person] = dfs[person].append(newDf, ignore_index=True)


{'pranav':                                              input_text  \
 0                         gym routine ill send u my ppl   
 1     gym everything is 3x8 unless stated otherwise ...   
 2     if ur by yourself do bench incline shoulder on...   
 3                                              benchkis   
 4              u have his number give him the uw wisdom   
 ...                                                 ...   
 1037  datum this message responded to an earlier mes...   
 1038  yeah this is bigger than i thought lowkey like...   
 1039  nah id win b yeah this is bigger than i though...   
 1040  nah id win b this message responded to an earl...   
 1041  dayum i think u did show me bro hw bag is like...   
 
                                             output_text  
 0     reactions loved by me yo whats our usual routine   
 1                                                   ok   
 2                                                   ty   
 3                              

In [7]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import nltk

In [8]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

batch_size = 2
block_size = 128
max_iters = 200
learning_rate = 3e-4
eval_iters = 100
n_embd = 128
n_head = 4
n_layer = 2
dropout = 0.2

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)

        # 2 Decoder and Encoder Layers
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm (make it normalized)
        self.lm_head = nn.Linear(n_embd, vocab_size) # Linear layer -> to make it softmax workable and led to prob. dist
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index


cpu
